<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/%E6%80%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

In [ ]:
#改名

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

In [ ]:
#在这说明那个调用的获取两个文件的事情。

***Visualising the comparison of different results is a good way to justify your decision.***

[XXX] = 讲座/实验参考
说明你的决定）= 请在文件中说明你的决定/选择的理由。你必须在报告中用经验性的证据表明你的最终决定。
解释性能）= 请解释性能的趋势，以及为什么会出现这样的趋势的原因（或你的意见）。

In [ ]:
#必须要有来自哪个lab

# 1 - Data Preprocessing

## 1.1. Download Dataset

If you want to know how data has been saved in pickle file: see this [ipynb file](https://drive.google.com/file/d/1ZQUVBzgH7N2EbiyE3WTPx7JNe2eRTs36/view?usp=sharing)

In [2]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle
training_data = pickle.load(open("training_data.pkl","rb"))
testing_data = pickle.load(open("testing_data.pkl","rb"))

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data)))
print("Size of testing dataset: {0}".format(len(testing_data)))
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")


------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


## 1.2. Preprocess data



*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *

**说明你的想法**

In [3]:
'''
# Please comment your code 评述你的代码 NLTK's Twitter_Sample dataset. 
# (Justify your decision) #[lab5]
'''
#[lab5]
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#分集合;因为是给定的数据集都是数据+标签的形式，所以要把它分开

train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]

test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos

# 随机训练集 为了让模型训练的更好，因为给的训练集前面是前后分开的000111
#看起来不加也行，加了准确率就提升了。。；没加之前是得根据batch_size 来调整训练集，要不然会出现训练模型时候，准确度0.5/1恒定的情况。 虽然loss在变，但是分类的情况却没有变。
#[lab3]
zipped = zip(train_data,train_label)   #上图说明 b 调用一次之后，再调用的 c 则为空。如果想要复用这个结果，需要保存到 list 里，即：
Zipp = list(zipped) #如果直接 print(list(zipped))，则之后 zipped为空，因为已经用完一次迭代器了；如果想要复用这个结果，需要保存到 list 里
from random import shuffle

shuffle(Zipp)
train_data = [context[0] for context in Zipp]
train_label = [context[1] for context in Zipp]


#做预处理 变小写 [lab5] 因为小写的话更容易匹配字典中的元素。就不会出现有些小写有些大写的情况，让数据集更加工整，也为了后续处理删词什么的方便。
import pprint
text_train = [s.lower() for s in train_data]
text_test = [s.lower() for s in test_data]

# 分词[lab5] 因为就是需要一个个词的来处理。通过一个或者多个词来判断整个句子的关系，那么首先要判断每个词的关系。
import pprint
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()


train_t=[]  #完全每句话的分词的集合。train_t[0][0]每个单词
for s in text_train:
  text_train = tknzr.tokenize(s)              #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
        #:(    :-(


test_t=[]     #2000
for w in text_test:
  text_test = tknzr.tokenize(w)
  test_t.append(text_test)

# 删标点，这里是保留了一部分表情。但是后来发现，因为字典中没有表情，所以可以不保留表情[lab5]
#theprincesszooz but i see what youre going at   
#yes  subjective pain may not be real
#  but that does not make it less painful
'''
还得改
clean_doc2 = re.sub(r'[^\w\s]','',corpus[1])字符
clean_doc2 = re.sub(r'\d,'',corpus[1])数字
'''
import re
def remove(x):
  t = []
  for i in range(len(x)):
    t_sub = []        #是直接用空列表代替了
    for j in range(len(x[i])):
      if len(x[i][j])==0:
        continue
      if x[i][j] == " ":
        continue 
      else:
        x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
        if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
          x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
        if len(x[i][j]) == 0:
          continue 
        else:
          t_sub.append(x[i][j]) 
    t.append(t_sub) 

  return t

text_train = remove(train_t)
text_test = remove(test_t)

##停用词  删除那些可有可无不影响句意比如a;an;the 但是又数目非常多的词。
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()
sww = sw.words()

text_train_le=[]    #8000
for tokens in text_train:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_le.append(filtered_sentence1)


text_test_le=[]   #2000
for tokens in text_test:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_le.append(filtered_sentence2)

#去掉复数 stemmer 词根不需要这部； 如果用就需要在neg和pos需要stemmer操作 保留是因为更加便捷。
#如果只保留原型lec…………那个，无法与现有给定字典一一匹配。

from nltk.stem.porter import *
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
train_stem = []
for i in range(len(text_train_le)):
  singles1 = []
  for plural in text_train_le[i]:
    singles1.append(stemmer.stem(plural))
  train_stem.append(singles1)

#print(train_stem[:5])

test_stem = []
for i in range(len(text_test_le)):
  singles2 = []
  for plural in text_test_le[i]:
    singles2.append(stemmer.stem(plural))
  test_stem.append(singles2)

#print(test_stem[:5])



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

我到底用了哪个

In [4]:
# Please comment your code
#是调用的gensim模型[lab2]
'''
还有好几个模型没做对比尝试呢
'''
#已经在上面预处理完了，然后就是建立模型。
from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences= train_stem + test_stem, size=100, window=5, min_count=2, workers=2, sg=0)
#训练模型
#看看单词
vocab = list(set(wv_cbow_model.wv.vocab.keys()))
print('vocabulary:', len(vocab), vocab[:5])

#找出某个词向量最相近的词集合。
req_count = 100
for key in wv_cbow_model.wv.similar_by_word('school', topn =100):   #'school'.decode("ISO-8859-1")
    if len(key[0])==5:  #最相近的五个字的词
        req_count -= 1
        print(key[0], key[1])
        if req_count == 0:
            break;

#查看相似度。看两个词的相似度
print(wv_cbow_model.wv.similarity('thing', 'hope'))
print(wv_cbow_model.wv.similarity('phone', 'sleep'))

#训练
wv_cbow_model.save("cbow.model")
#训练 ； 存 取；训练。
cbow = Word2Vec.load("./cbow.model")  #只是训练的NLTK's Twitter_Sample dataset. cbow最原始的标记方法。
#再训练epochs训练多少epoch ##end_alpha ==learning rate final;   total_examples : count of sentences.10000
ep = 0
for i in range(10):
  cbow.train(sentences= train_stem + test_stem ,total_examples=wv_cbow_model.corpus_count, epochs=cbow.iter, compute_loss=True)
  #举个例子，已经训练好的word2vec词表有[“我”,“爱”,“你”]三个词，给定一个新语料[“我”,“喜”,“欢”,“你”]，新语料只会调整原有词中的“我”“你”的词向量表示，而不会将“喜”“欢”添加到词库中。
  ep += cbow.epochs
  print('Epoch: %d, train loss: %.5f'%(ep, cbow.get_latest_training_loss()))

vocabulary: 4327 ['sir', 'repost', 'tabinda_samar', 'chat', 'ahead']
thing 0.9998672008514404
watch 0.9998614192008972
gonna 0.9998530149459839
still 0.9998481273651123
first 0.999843955039978
right 0.9998433589935303
peopl 0.9998365640640259
anoth 0.9998319149017334
there 0.9998281002044678
littl 0.9998272657394409
never 0.9998247623443604
music 0.9998242259025574
video 0.9998210668563843
wanna 0.999817967414856
could 0.999816358089447
alway 0.9998159408569336
think 0.9998087286949158
today 0.9998081922531128
phone 0.9998078346252441
0.99989986
0.99976516


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Epoch: 5, train loss: 350616.84375
Epoch: 10, train loss: 334654.25000
Epoch: 15, train loss: 320123.78125
Epoch: 20, train loss: 306175.90625
Epoch: 25, train loss: 284001.65625
Epoch: 30, train loss: 278337.12500
Epoch: 35, train loss: 273414.75000
Epoch: 40, train loss: 249550.78125
Epoch: 45, train loss: 227826.43750
Epoch: 50, train loss: 221944.10938


### 2.1.1. Data Preprocessing for Word Embeddings

*You are required to describe which preprocessing techniques were used with justification of your decision.*

In [5]:
# Please comment your code

### 2.1.2. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [6]:
# Please comment your code

### 2.1.3. Train Word Embeddings Model

In [7]:
# Please comment your code

## 2.2. Lexicon Embeddings

### 2.2.1. Lexicon-based Embedding

*[Optional] You are required to describe why you would like to use more than one-dimensional embedding.*

我认为是为了包含更多的信息也就是特征，更加准确的去得到你输入的和你想要结果之间的某种关系。

In [8]:
# Please comment your code
#否定词"ISO-8859-1"==UTF-8
'''
这里需要具体说明把文件丢到哪里
'''
#核心想法，我去看了看这个文件，因为有一些文本介绍。我找到了每个文件的第一个词，然后把这些词重新取出来放到一个list中。
from google.colab import drive
drive.mount('/content/drive') #必须要挂载
not_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件
j = -1
for i in not_words:
  j +=1
  if i == '2-faced':
    break
neg = not_words[j:]

#处理文件
sure_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt', 'r', encoding="ISO-8859-1").readlines()]

j = -1
for i in sure_words:
  j +=1
  if i == 'a+':
    break

pos = sure_words[j:]

#neg pos 词表
#要对他们做stem因为训练集做了stem

'''
前面要是后来测试发现不需要这里那就不用了。
或者存个模型。
'''
#[lab5]
#也要对他做stem #太久了。简直了 至少10-20分钟，太久了。 #该跑还是要跑。

neg_stem = []
for i in range(len(neg)):
  singles3 = []
  for plural in neg:
    singles3.append(stemmer.stem(plural))
  neg_stem.append(singles3)

print(neg_stem[:5])


pos_stem = []
for i in range(len(pos)):
  singles4 = []
  for plural in pos:
    singles4.append(stemmer.stem(plural))
  pos_stem.append(singles4)

print(pos_stem[:5])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'\nneg_stem = []\nfor i in range(len(neg)):\n  singles3 = []\n  for plural in neg:\n    singles3.append(stemmer.stem(plural))\n  neg_stem.append(singles3)\n\nprint(neg_stem[:5])\n\n\npos_stem = []\nfor i in range(len(pos)):\n  singles4 = []\n  for plural in pos:\n    singles4.append(stemmer.stem(plural))\n  pos_stem.append(singles4)\n\nprint(pos_stem[:5])\n'

In [9]:
#把每次训练的给存了起来。 
#neg_stem.save("neg_stem")  #AttributeError: 'list' object has no attribute 'save'
#pos_stem.save("pos_stem")
#
'''
1.我们先将需要保存的数据解析好，保存成固定的数据类型（保存成列表，元组，字典都可以，根据具体场景来选择）

2.我们将保存数据到csv文件的代码封装成一个函数，方便重用。步骤主要分为三步：打开文件，写入数据，关闭文件。其中，写入数据时记得先写入表头（我们使用excel打开时需要表头）再写入表格中的数据，数据要以一个列表的形式传递给writerows()。

运行结果：

运行以上代码后，会在当前目录下创建一个csv_file.csv的文件，并写入csv_data的数据，可以使用excel打开文件查看。如下图。
1.可以通过with上下文管理的方式打开csv文件，如果想在with的代码块外对读出来的数据进行处理，则可以使用open()打开，再使用close()关闭。

2.csv通过csv.reader()来打开csv文件，返回的是一个列表格式的迭代器，可以通过next()方法获取其中的元素，也可以使用for循环依次取出所有元素。


'''

'\n1.我们先将需要保存的数据解析好，保存成固定的数据类型（保存成列表，元组，字典都可以，根据具体场景来选择）\n\n2.我们将保存数据到csv文件的代码封装成一个函数，方便重用。步骤主要分为三步：打开文件，写入数据，关闭文件。其中，写入数据时记得先写入表头（我们使用excel打开时需要表头）再写入表格中的数据，数据要以一个列表的形式传递给writerows()。\n\n运行结果：\n\n运行以上代码后，会在当前目录下创建一个csv_file.csv的文件，并写入csv_data的数据，可以使用excel打开文件查看。如下图。\n1.可以通过with上下文管理的方式打开csv文件，如果想在with的代码块外对读出来的数据进行处理，则可以使用open()打开，再使用close()关闭。\n\n2.csv通过csv.reader()来打开csv文件，返回的是一个列表格式的迭代器，可以通过next()方法获取其中的元素，也可以使用for循环依次取出所有元素。\n\n\n'

In [ ]:
#把列表存进去
'''
import csv
neg_name = 'neg.csv'
pos_name = 'pos.csv'
def save_csv(target_list, output_file_name):
    """
    将数据写入csv文件
    """
    if not output_file_name.endswith('.csv'):
        output_file_name += '.csv'
    neg = open(output_file_name, "w", newline="")
    key_data = target_list[0]
    value_data = [target for target in target_list]
    csv_writer = csv.writer(neg)
    csv_writer.writerow(key_data)
    csv_writer.writerows(value_data)
    neg.close()
 
 
save_csv(neg_stem, neg_name)
save_csv(pos_stem, pos_name)

input_file_name1 = 'neg.csv'
input_file_name2 = 'pos.csv'

with open(input_file_name1, 'r', encoding='ISO-8859-1') as csv_file:
    csv_reader = csv.reader(csv_file)
    neg = next(csv_reader)    #neg == neg_stem

with open(input_file_name2, 'r', encoding='ISO-8859-1') as csv_file:
    csv_reader = csv.reader(csv_file)
    pos = next(csv_reader)    #  pos ==pos_stem

    '''

In [ ]:
'''

import csv
import codecs

def data_write_csv(file_name, datas):#file_name为写入CSV文件的路径，datas为要写入数据列表
          file_csv = codecs.open(file_name,'w+','utf-8')#追加
          writer = csv.writer(file_csv, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
          for data in datas:
              writer.writerow(data)
          print("保存文件成功，处理结束")
data_write_csv()
'''


 
'''
csv_data = (
    (1, 2, 3, 4, 5, 6),
    ('a', 'b', 'c', 'd', 'e', 'f'),
    ('p', 'y', 't', 'h', 'o', 'n')
)
'''

import csv
neg_name = 'neg.csv'
 
 
def save_csv(target_list, output_file_name):
    """
    将数据写入csv文件
    """
    if not output_file_name.endswith('.csv'):
        output_file_name += '.csv'
    neg = open(output_file_name, "w", newline="")
    key_data = target_list[0]
    value_data = [target for target in target_list]
    csv_writer = csv.writer(neg)
    csv_writer.writerow(key_data)
    csv_writer.writerows(value_data)
    neg.close()
 
 
save_csv(neg_stem, neg_name)


input_file_name = 'neg.csv'
 

with open(input_file_name, 'r', encoding='ISO-8859-1') as csv_file:
    csv_reader = csv.reader(csv_file)
    neg = next(csv_reader)
    # csv_reader对象，是一个列表的格式
    #print(csv_reader)
    # csv_reader对象的一个迭代器，可以通过next()取出其中的元素
    #print(next(csv_reader))
    # 也可以通过for循环取出所有元素
    #for line in csv_reader:
        #print(''.join(line))
        

''' 
def read_csv(input_file_name):
    """
    读取csv文件数据
    """
    with open(input_file_name, 'r', encoding='ISO-8859-1') as csv_file:
        csv_reader = csv.reader(csv_file)
        # csv_reader对象，是一个列表的格式
        print(csv_reader)
        # csv_reader对象的一个迭代器，可以通过next()取出其中的元素
        print(next(csv_reader))
        # 也可以通过for循环取出所有元素
        #for line in csv_reader:
            #print(''.join(line))
 
 
neg = read_csv(neg_name)
'''

#https://blog.csdn.net/weixin_43790276/article/details/90572596?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161941924416780269863927%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161941924416780269863927&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_positive~default-1-90572596.first_rank_v2_pc_rank_v29&utm_term=csv


In [ ]:
'''
import csv
neg_file_name = 'neg_stem.csv'


def save_csv(target_list, output_file_name):
  #将数据写入CSV
  if not output_file_name.endswith('.csv'):
    output_file_name +='.csv'
  csv_file = open(output_file_name, "w",newline="") #来控制空行数。
  key_data = target_list[0]
  value_data = [target for target in target_list]
  csv_writer = csv.writer(csv_file) #创建一个csv的写入器
  csv_writer.writerow(key_data) #写入标签
  csv_writer.writerows(value_data)  #写入样本数据
  csv_file.close()

save_csv(neg_stem, neg_file_name)
save_csv(pos_stem, pos_file_name)

'''

In [ ]:
#字典的embedding ； 增加一个维度的 
#只需要先让他们对应起来。您要检查每个单词是在正还是负词典中not_exist（0），negative（1）和positive（2）
#一维句子，二维单词
#是要吧一句话变成【012组成的形式】
#现在就是一一对应的形式了。一句话;这个是train_stem【 as,ada,daff】 match_embedding【120】同一个索引在两个列表中 
def match(text):
  sum_match= [] #train_match
  for i in range(len(text)):  #train_stem
    match = []
    for j in range(len(text[i])):
      if text[i][j] in neg_stem[0]:
        match.append(1)   #neg 1
      elif text[i][j] in pos_stem[0]:
        match.append(2) #pos 2
      else:
        match.append(0)
    sum_match.append(match)
  #print(sun_match[:5])
  #print(text[:5]) 
  return sum_match
train_match = match(train_stem)
test_match = match(test_stem)
#只有stem好用
print(train_match[:5])
print(test_match[:5])

## 2.3. Bi-directional RNN Sequence model

### 2.3.1. Apply/Import Word Embedding and Lexicon Embedding

把词嵌入和字典嵌入连接起来。

In [ ]:
# Please comment your code


### 2.3.2. Build Sequence Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [ ]:
# Please comment your code

### 2.3.3. Train Sequence Model

Note that it will not be marked if you do not display the Training Loss and the Number of Epochs in the Assignment 1 ipynb.


In [ ]:
# Please comment your code

# 3 - Evaluation


## 3.1. Word Embedding Evaluation
You are to apply Semantic-Syntactic word relationship tests for the trained word embeddings and visualise the result of Semantic-Syntactic word relationship tests.
Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence*)

In [ ]:
# Please comment your code

## 3.2. Performance Evaluation


You are required to provide the table with precision, recall, f1 of test set.
Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence*)

In [ ]:
# Please comment your code

## 3.3. Hyperparameter Testing
*You are required to draw a graph(y-axis: f1, x-axis: epoch) for test set and explain the optimal number of epochs based on the learning rate you have already chosen.* Note that it will not be marked if you do not display it in the ipynb file.


(*Please show your empirical evidence*)

In [ ]:
# Please comment your code

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed* 

In [ ]:
# If you used OOP style, use this section